In [2]:
import pandas as pd

In [3]:
used_car= pd.read_csv('gs://used_car/data/used_cars_data_sample.csv')

/tmp/ipykernel_3174/2978110390.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  used_car= pd.read_csv('gs://used_car/data/used_cars_data_sample.csv')


In [23]:
used_car.shape

(750010, 67)

In [4]:
3000040/4

750010.0

In [5]:
used_car.head()

,vin,back_legroom,bed,bed_height,bed_length,body_type,cabin,city,city_fuel_economy,combine_fuel_economy,...,transmission,transmission_display,trimId,trim_name,vehicle_damage_category,wheel_system,wheel_system_display,wheelbase,width,year
0,ZACNJABB5KPJ92081,35.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,Bayamon,NaN,NaN,...,A,9-Speed Automatic Overdrive,t83804,Latitude FWD,NaN,FWD,Front-Wheel Drive,101.2 in,79.6 in,2019
1,SALCJ2FX1LH858117,38.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,...,A,9-Speed Automatic Overdrive,t86759,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020
2,JF1VA2M67G9829723,35.4 in,NaN,NaN,NaN,Sedan,NaN,Guaynabo,17.0,NaN,...,M,6-Speed Manual,t58994,Base,NaN,AWD,All-Wheel Drive,104.3 in,78.9 in,2016
3,SALRR2RV0L2433391,37.6 in,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,...,A,8-Speed Automatic Overdrive,t86074,V6 HSE AWD,NaN,AWD,All-Wheel Drive,115 in,87.4 in,2020
4,SALCJ2FXXLH862327,38.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,...,A,9-Speed Automatic Overdrive,t86759,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020


In [24]:
used_car.dtypes

Unnamed: 0               int64
vin                     object
back_legroom            object
bed                     object
bed_height              object
                         ...  
wheel_system            object
wheel_system_display    object
wheelbase               object
width                   object
year                     int64
Length: 67, dtype: object

In [13]:
used_car.head(750010).to_csv("gs://used_car/data/used_cars_data_sample.csv")

In [15]:
used_car.head(100).to_csv("gs://used_car/data/used_cars_data_sample100.csv")

In [5]:
%%writefile testutility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re
################
# File Reading #
################

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string
def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Writing testutility.py


In [6]:
!ls

bin	   dev	   home   lost+found  opt   run   sys		  usr
boot	   etc	   lib	  media       proc  sbin  testutility.py  var
copyright  hadoop  lib64  mnt	      root  srv   tmp


In [7]:
import testutility as util

In [9]:
%%writefile file.yaml
file_type: csv
bucket: used_car
fold_name: data
file_name: used_cars_data_sample
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
    - vin
    - bed
    - year

Overwriting file.yaml


In [10]:
# Read config file
import testutility as util
config_data = util.read_config_file("file.yaml")

In [11]:
#inspecting data of config file
config_data

{'file_type': 'csv',
 'bucket': 'used_car',
 'fold_name': 'data',
 'file_name': 'used_cars_data_sample',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': ['vin', 'bed', 'year']}

In [12]:
#inspecting data of config file
config_data['file_name']

'used_cars_data_sample'

In [ ]:
"gs://used_car/data/used_cars_data_sample.csv"

In [17]:
# read the file using config file
file_type = config_data['file_type']
source_file = "gs://" + config_data['bucket'] +'/'+ config_data['fold_name']+'/'+config_data['file_name']+'.'+ config_data['file_type']
#print("",source_file)
df = pd.read_csv(source_file,delimiter=config_data['inbound_delimiter'])
df.head()

/tmp/ipykernel_3174/3412405820.py:5: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(source_file,delimiter=config_data['inbound_delimiter'])


,Unnamed: 0,vin,back_legroom,bed,bed_height,bed_length,body_type,cabin,city,city_fuel_economy,...,transmission,transmission_display,trimId,trim_name,vehicle_damage_category,wheel_system,wheel_system_display,wheelbase,width,year
0,0,ZACNJABB5KPJ92081,35.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,Bayamon,NaN,...,A,9-Speed Automatic Overdrive,t83804,Latitude FWD,NaN,FWD,Front-Wheel Drive,101.2 in,79.6 in,2019
1,1,SALCJ2FX1LH858117,38.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,...,A,9-Speed Automatic Overdrive,t86759,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020
2,2,JF1VA2M67G9829723,35.4 in,NaN,NaN,NaN,Sedan,NaN,Guaynabo,17.0,...,M,6-Speed Manual,t58994,Base,NaN,AWD,All-Wheel Drive,104.3 in,78.9 in,2016
3,3,SALRR2RV0L2433391,37.6 in,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,...,A,8-Speed Automatic Overdrive,t86074,V6 HSE AWD,NaN,AWD,All-Wheel Drive,115 in,87.4 in,2020
4,4,SALCJ2FXXLH862327,38.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,...,A,9-Speed Automatic Overdrive,t86759,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020


In [20]:
#validate the header of the file
util.col_header_val(df,config_data)

column name and column length validation failed
Following File columns are not in the YAML file ['engine_cylinders', 'has_accidents', 'owner_count', 'bed_length', 'fuel_tank_volume', 'city', 'main_picture_url', 'longitude', 'seller_rating', 'theft_title', 'transmission_display', 'trimid', 'combine_fuel_economy', 'horsepower', 'major_options', 'body_type', 'width', 'latitude', 'listed_date', 'is_new', 'is_oemcpo', 'fuel_type', 'description', 'franchise_dealer', 'make_name', 'unnamed_0', 'listing_color', 'salvage', 'listing_id', 'height', 'price', 'savings_amount', 'transmission', 'interior_color', 'city_fuel_economy', 'model_name', 'frame_damaged', 'front_legroom', 'engine_type', 'wheelbase', 'cabin', 'bed_height', 'daysonmarket', 'is_certified', 'maximum_seating', 'highway_fuel_economy', 'trim_name', 'sp_name', 'fleet', 'iscab', 'sp_id', 'length', 'exterior_color', 'mileage', 'power', 'franchise_make', 'vehicle_damage_category', 'engine_displacement', 'torque', 'wheel_system', 'back_le

0

In [21]:
print("columns of files are:" ,df.columns)
print("columns of YAML are:" ,config_data['columns'])

columns of files are: Index(['unnamed_0', 'vin', 'back_legroom', 'bed', 'bed_height', 'bed_length',
       'body_type', 'cabin', 'city', 'city_fuel_economy',
       'combine_fuel_economy', 'daysonmarket', 'dealer_zip', 'description',
       'engine_cylinders', 'engine_displacement', 'engine_type',
       'exterior_color', 'fleet', 'frame_damaged', 'franchise_dealer',
       'franchise_make', 'front_legroom', 'fuel_tank_volume', 'fuel_type',
       'has_accidents', 'height', 'highway_fuel_economy', 'horsepower',
       'interior_color', 'iscab', 'is_certified', 'is_cpo', 'is_new',
       'is_oemcpo', 'latitude', 'length', 'listed_date', 'listing_color',
       'listing_id', 'longitude', 'main_picture_url', 'major_options',
       'make_name', 'maximum_seating', 'mileage', 'model_name', 'owner_count',
       'power', 'price', 'salvage', 'savings_amount', 'seller_rating', 'sp_id',
       'sp_name', 'theft_title', 'torque', 'transmission',
       'transmission_display', 'trimid', 'trim_nam

In [22]:
if util.col_header_val(df,config_data)==0:
    print("validation failed")
    # write code to reject the file
else:
    print("col validation passed")
    # write the code to perform further action
    # in the pipleine

column name and column length validation failed
Following File columns are not in the YAML file ['engine_cylinders', 'has_accidents', 'owner_count', 'bed_length', 'fuel_tank_volume', 'city', 'main_picture_url', 'longitude', 'seller_rating', 'theft_title', 'transmission_display', 'trimid', 'combine_fuel_economy', 'horsepower', 'major_options', 'body_type', 'width', 'latitude', 'listed_date', 'is_new', 'is_oemcpo', 'fuel_type', 'description', 'franchise_dealer', 'make_name', 'unnamed_0', 'listing_color', 'salvage', 'listing_id', 'height', 'price', 'savings_amount', 'transmission', 'interior_color', 'city_fuel_economy', 'model_name', 'frame_damaged', 'front_legroom', 'engine_type', 'wheelbase', 'cabin', 'bed_height', 'daysonmarket', 'is_certified', 'maximum_seating', 'highway_fuel_economy', 'trim_name', 'sp_name', 'fleet', 'iscab', 'sp_id', 'length', 'exterior_color', 'mileage', 'power', 'franchise_make', 'vehicle_damage_category', 'engine_displacement', 'torque', 'wheel_system', 'back_le

In [25]:
df.to_csv('gs://used_car/data/used_car_sample.txt.gz', sep='|', compression='gzip', index=False)